In [1]:
import fastkaggle
import polars as pl

comp = "playground-series-s4e5"
path = fastkaggle.setup_comp(comp)

In [2]:
train = pl.read_csv(path/"train.csv")
test = pl.read_csv(path/"test.csv")

In [3]:
from sklearn.ensemble import HistGradientBoostingClassifier, HistGradientBoostingRegressor

In [4]:
train.select(std=pl.col("FloodProbability").std(), mean=pl.col("FloodProbability").mean()) 

std,mean
f64,f64
0.051026,0.50448


In [5]:
train.columns

['id',
 'MonsoonIntensity',
 'TopographyDrainage',
 'RiverManagement',
 'Deforestation',
 'Urbanization',
 'ClimateChange',
 'DamsQuality',
 'Siltation',
 'AgriculturalPractices',
 'Encroachments',
 'IneffectiveDisasterPreparedness',
 'DrainageSystems',
 'CoastalVulnerability',
 'Landslides',
 'Watersheds',
 'DeterioratingInfrastructure',
 'PopulationScore',
 'WetlandLoss',
 'InadequatePlanning',
 'PoliticalFactors',
 'FloodProbability']

In [6]:
X = train.select(pl.all().exclude("id", "FloodProbability"))
y = train.get_column("FloodProbability").to_numpy()

In [10]:
from sklearn.model_selection import cross_validate, KFold

cross_validate(HistGradientBoostingRegressor(loss="gamma"), X, y, cv=KFold(), scoring="r2")

{'fit_time': array([7.66179204, 7.2624042 , 7.27417588, 6.79349875, 6.82809329]),
 'score_time': array([0.39700198, 0.4000361 , 0.40024972, 0.39639878, 0.39331198]),
 'test_score': array([0.76650382, 0.76484852, 0.76637766, 0.76670384, 0.76565526])}

In [ ]:
mod = HistGradientBoostingRegressor()
mod.fit(X,y)

In [ ]:
test =pl.read_csv(path/"test.csv")

In [ ]:
test.head(3)

In [ ]:
X_test = test.drop("id")
preds = mod.predict(X_test)

In [ ]:
sub = pl.read_csv(path/"sample_submission.csv")
sub

In [ ]:
sub = sub.with_columns(FloodProbability=preds)

In [ ]:
sub.write_csv(path/"subm.csv")
sub.head(2)

In [ ]:
from kaggle import api
api.competition_submit_cli(path/'subm.csv', 'boost-0-feature-eng', comp)

In [ ]:
cross_validate(HistGradientBoostingClassifier(), X, y>0.5, cv=KFold(), scoring="accuracy")

In [ ]:
mod = HistGradientBoostingClassifier()
mod.fit(X[:900000],y[:900000]>0.5)

In [ ]:
pred_proba = mod.predict_proba(X[900000:])

In [ ]:
preds = mod.predict(X[900000:])

In [ ]:
preds

In [ ]:
from sklearn.metrics import r2_score

r2_score(pred_proba[:,1], y[900000:])

In [ ]:
r2_score(preds, y[900000:])

In [ ]:
X.plot.kde()

In [ ]:
train.select(pl.last()).plot.hist()

In [ ]:
X.select(pl.first()).plot.hist()

In [ ]:
test.drop("id").select(pl.first()).plot.hist()

In [ ]:
train.drop("id").select(pl.all().exclude("FloodProbability").n_unique())

In [ ]:
test.drop("id").select(pl.all().exclude("FloodProbability").n_unique())

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor
HistGradientBoostingRegressor(scoring="r2")

In [22]:
from ast import expr


X_ss = X.select(pl.reduce(function = lambda acc,x: acc+x, exprs=pl.all()).alias("sum"))

In [23]:
from sklearn.linear_model import Ridge

In [25]:
cross_validate(Ridge(), X_ss, y, cv=KFold(), scoring="r2")

{'fit_time': array([0.2519443 , 0.1340909 , 0.15849495, 0.16112447, 0.16423726]),
 'score_time': array([0.00309849, 0.00346732, 0.00334716, 0.00368381, 0.00342298]),
 'test_score': array([0.84511278, 0.84335424, 0.84461139, 0.84659108, 0.84498482])}

In [26]:
test = pl.read_csv(path/"test.csv").drop("id")

In [28]:
test_ss =  test.select(pl.reduce(function = lambda acc,x: acc+x, exprs=pl.all()).alias("sum"))

In [29]:
mod = Ridge().fit(X_ss,y)

In [31]:
preds = mod.predict(test_ss)

In [33]:
import numpy as np
np.sum(preds>1)+np.sum(preds<0)

0

In [41]:
sub = pl.read_csv(path/"sample_submission.csv")
sub = sub.select("id", FloodProbability=preds)

In [42]:
sub

id,FloodProbability
i64,f64
1117957,0.573429
1117958,0.454861
1117959,0.454861
1117960,0.466154
1117961,0.466154
…,…
1863257,0.477446
1863258,0.449215
1863259,0.624243


In [43]:
sub.write_csv(path/"subm.csv")

In [45]:
from kaggle import api

In [46]:
api.competition_submit_cli(path/'subm.csv', 'sum_feat', comp)

100%|██████████| 19.3M/19.3M [00:09<00:00, 2.24MB/s]


Successfully submitted to Regression with a Flood Prediction Dataset